# Рекомендация тарифов

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

### Оглавление
[1. Откройте и изучите файл](#open_file)  
[2. Разбейте данные на выборки](#split_data)  
[3. Исследуйте модели](#explore_models)  
[4. Проверьте модель на тестовой выборке](#test_models)  
[5. (бонус) Проверьте модели на адекватность](#adequacy_models)  
[Вывод](#conclusion)  

<a id="open_file"></a>
## Откройте и изучите файл

#### Импортируем библиотеки

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
import numpy as np
from sklearn.dummy import DummyClassifier
import warnings
from IPython.display import display
warnings.filterwarnings("ignore", category=FutureWarning)

#### Изучим файл с данными

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [4]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [5]:
df.isna().sum()

calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

In [6]:
df.duplicated().sum()

0

### Вывод

Таблица состоит из 5 колонок и 3214 строк, пропуски и дубликаты отсутствуют.  
Известны следующие параметры:
- *сalls* — количество звонков,
- *minutes* — суммарная длительность звонков в минутах,
- *messages* — количество sms-сообщений,
- *mb_used* — израсходованный интернет-трафик в Мб,
- *is_ultra* — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

<a id="split_data"></a>
## Разбейте данные на выборки

Для дальнейшего создания моделей требуется выделить **целевой признак**.  
В нашем случае целевым признаком является столбец **is_ultra**.

In [7]:
# извлечем признаки 
features = df.drop('is_ultra', axis=1)

In [8]:
# извлечем целевой признак
target = df['is_ultra']

В случае, когда необходимо выделить тестовую выборку из всего набора данных применяют соотношение:  
- обучающая - 60 %,
- валидационная - 20%,
- тестовая - 20%.

#### Выделим из данных тестовую выборку методом `train_test_split`:

In [9]:
# отделим 20% данных для тестовой выборки
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.20, random_state=12345)

In [10]:
print(features.shape)
print(features_train.shape)
print(features_test.shape)

(3214, 4)
(2571, 4)
(643, 4)


#### Выделим из данных валидатационную выборку методом `train_test_split`:

In [11]:
# отделим 25% данных для валидатационной выборки
features_train, features_valid, target_train, target_valid = train_test_split(
    features_train, target_train, test_size=0.25, random_state=12345)

In [12]:
features_valid.shape

(643, 4)

**Проверим сходимость изначального размера таблицы:**

In [13]:
(features_train + features_valid + features_test).shape

(3214, 4)

### Вывод

Мы выделили целевой признак в таблице - столбец **is_ultra**.  
Разбили данные на 3 выборки (в соотношении 3:1:1): **обучающую, валидационную, тестовую.**

<a id="explore_models"></a>
## Исследуйте модели

Целевой признак является **категориальным и двоичным(значение 0 или 1)**, следовательно решается задача *бинарной классификации*.  
Исходя из этих предположений, выберем модели работающие с задачами классификации:

#### Решающее дерево

Обучим модель на тренировочной выборке и подберем наиболее подходящую величину **гипрепараметра** - **глубина дерева(`max_depth`)**.  
Получим предсказания модели на валидатационной выборке и посчитаем качество модели `accuracy_score` для наиболее подходящего значения `max_depth`.

In [14]:
best_model_dts = None
best_result_dts = 0
for depth in range(1, 10):
    model_dts = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model_dts.fit(features_train, target_train)
    predictions_valid_dts = model_dts.predict(features_valid)
    result_dts = accuracy_score(target_valid, predictions_valid_dts)
    if result_dts > best_result_dts:
        best_model_dts = model_dts
        best_result_dts = result_dts
        best_depth = depth
        
print("Accuracy наилучшей модели на валидационной выборке:", best_result_dts)
print("Лучшее значение max_depth:", best_depth)

Accuracy наилучшей модели на валидационной выборке: 0.7744945567651633
Лучшее значение max_depth: 7


In [15]:
for depth in range(1, 10):
    model_dts = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model_dts.fit(features_train, target_train)
    predictions_valid_dts = model_dts.predict(features_valid)
    result_dts = accuracy_score(target_valid, predictions_valid_dts)
    print('max_depth = {} : {}'.format(depth, result_dts))

max_depth = 1 : 0.7387247278382582
max_depth = 2 : 0.7573872472783826
max_depth = 3 : 0.7651632970451011
max_depth = 4 : 0.7636080870917574
max_depth = 5 : 0.7589424572317263
max_depth = 6 : 0.7573872472783826
max_depth = 7 : 0.7744945567651633
max_depth = 8 : 0.7667185069984448
max_depth = 9 : 0.7620528771384136


Как мы видим, accuracy наилучшей модели **"Решающее дерево"** достигается при **глубине дерева равной 7.**

#### Случайный лес

Обучим модель на тренировочной выборке и подберем наиболее подходящую величину **гипрепараметров**:  
1. **количество деревьев(`n_estimators`)**.  
2. **макисимальная глубина деревьев(`max_depth`)**.
3. **число признаков(`max_features`)**.
Получим предсказания модели на валидатационной выборке и посчитаем качество модели `score` для наиболее подходящих значений гиперпараметров.

In [16]:
est_model_rfc = None
best_result_rfc = 0
for est in range(1, 6):
    for depth in range(1, 6):
        for mf in range(1, 5):
            model_rfc = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth, max_features=mf)
            model_rfc.fit(features_train, target_train) 
            result_rfc = model_rfc.score(features_valid, target_valid) 
            if result_rfc > best_result_rfc:
                best_model_rfc = model_rfc 
                best_result_rfc = result_rfc
                best_est = est
                best_depth = depth
                best_mf = mf
print("Accuracy наилучшей модели на валидационной выборке:", best_result_rfc)
print("Лучшее значение n_estimators", best_est)
print("Лучшее значение max_depth:", best_depth)
print("Лучшее значение max_features", best_mf)

Accuracy наилучшей модели на валидационной выборке: 0.7838258164852255
Лучшее значение n_estimators 5
Лучшее значение max_depth: 5
Лучшее значение max_features 3


**Рассмотрим отдельно каждый гиперпараметр перебором значений:**

In [17]:
# количество деревьев(n_estimators)
for est in range(1, 6):
    model_rfc = RandomForestClassifier(random_state=12345, n_estimators=est, max_features=mf) 
    model_rfc.fit(features_train, target_train) 
    result_rfc = model_rfc.score(features_valid, target_valid) 
    print('n_estimators = {} : {}'.format(est, result_rfc))

n_estimators = 1 : 0.7247278382581649
n_estimators = 2 : 0.7589424572317263
n_estimators = 3 : 0.7636080870917574
n_estimators = 4 : 0.7744945567651633
n_estimators = 5 : 0.7589424572317263


In [18]:
# макисимальная глубина деревьев(max_depth)
for depth in range(1, 6):
    model_rfc = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth, max_features=mf) 
    model_rfc.fit(features_train, target_train) 
    result_rfc = model_rfc.score(features_valid, target_valid) 
    print('depth = {} : {}'.format(depth, result_rfc))

depth = 1 : 0.7402799377916018
depth = 2 : 0.7682737169517885
depth = 3 : 0.7698289269051322
depth = 4 : 0.7744945567651633
depth = 5 : 0.7822706065318819


In [19]:
# число признаков(max_features)
for mf in range(1, 5):
    model_rfc = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth, max_features=mf)
    model_rfc.fit(features_train, target_train) 
    result_rfc = model_rfc.score(features_valid, target_valid) 
    print('features = {} : {}'.format(mf, result_rfc))

features = 1 : 0.7682737169517885
features = 2 : 0.7698289269051322
features = 3 : 0.7838258164852255
features = 4 : 0.7822706065318819


Accuracy наилучшей модели **"Случайный лес"** достигается при **количестве деревьев равном 5, максимальной глубине - 5, числу признаков - 3.**

#### Логистическая регрессия

Обучим модель на тренировочной выборке и получим предсказания модели на валидатационной выборке и посчитаем качество модели `score`.

In [20]:
model_lr = LogisticRegression(random_state=12345) 
model_lr.fit(features_train, target_train) 
result_lr = model_lr.score(features_valid, target_valid) 

print("Accuracy модели логистической регрессии на валидационной выборке:", result_lr)

Accuracy модели логистической регрессии на валидационной выборке: 0.6967340590979783


#### Отдельно выведем значение Accuracy для каждой модели

In [21]:
data= {best_result_dts, best_result_rfc, result_lr}
pd.DataFrame(data, index=('Решающее дерево', 'Случайный лес', 'Логистическая регрессия'), columns=['Accuracy'])

,Accuracy
Решающее дерево,0.774495
Случайный лес,0.783826
Логистическая регрессия,0.696734


### Вывод

Лучший результат ***Accuracy (точности)*** в **0.783826** показала модель построенная по алгортму машинного обучения ***Random forest (множество решающих деревьев)***.  
Наиболее оптимальные гиперпараметры для модели ***Random forest***:
- количество деревьев (n_estimators) - 4, 
- макисимальная глубина деревьев (max_depth) - 5, 
- число признаков (max_features) - 3.



<a id="test_models"></a>
## Проверьте модель на тестовой выборке

Проведем эксперименты для каждой модели на тестовых данных.

#### Решающее дерево

In [22]:
test_predictions_dts = best_model_dts.predict(features_test)
test_result_dts = accuracy_score(target_test, test_predictions_dts)
test_result_dts_norma = accuracy_score(target_test, test_predictions_dts, normalize=False)
print("Accuracy - Обучающая выборка:", best_result_dts)
print("Accuracy - Тестовая выборка:", test_result_dts)
print("Количество верных ответов:", test_result_dts_norma)

Accuracy - Обучающая выборка: 0.7744945567651633
Accuracy - Тестовая выборка: 0.7884914463452566
Количество верных ответов: 507


#### Случайный лес

In [23]:
test_predictions_rfc = best_model_rfc.predict(features_test)
test_result_rfc = accuracy_score(target_test, test_predictions_rfc)
test_result_rfc_norma = accuracy_score(target_test, test_predictions_rfc, normalize=False)
print("Accuracy - Обучающая выборка:", best_result_rfc)
print("Accuracy - Тестовая выборка:", test_result_rfc)
print("Количество верных ответов:", test_result_rfc_norma)

Accuracy - Обучающая выборка: 0.7838258164852255
Accuracy - Тестовая выборка: 0.7884914463452566
Количество верных ответов: 507


#### Логистическая регрессия

In [24]:
test_predictions_lr = model_lr.predict(features_test)
test_result_lr = accuracy_score(target_test, test_predictions_lr)
test_result_lr_norma = accuracy_score(target_test, test_predictions_lr, normalize=False)
print("Accuracy - Обучающая выборка:", result_lr)
print("Accuracy - Тестовая выборка:", test_result_lr)
print("Количество верных ответов:", test_result_lr_norma)

Accuracy - Обучающая выборка: 0.6967340590979783
Accuracy - Тестовая выборка: 0.702954898911353
Количество верных ответов: 452


In [25]:
data2= {'Accuracy_valid':[best_result_dts, best_result_rfc, result_lr], 
        'Accuracy_test': [test_result_dts, test_result_rfc, test_result_lr]}
pd.DataFrame(data2, index=('Решающее дерево', 'Случайный лес', 'Логистическая регрессия'))

,Accuracy_valid,Accuracy_test
Решающее дерево,0.774495,0.788491
Случайный лес,0.783826,0.788491
Логистическая регрессия,0.696734,0.702955


### Вывод

Все модели показали схожие результаты точности на тестовой выборке и при их обучении.  
Как и ожидалось, больше всего точных ответов предсказывают модели ***случайный лес (Random forest)*** и ***решающее дерево (Decision Tree).***  
Наименьшую точность показывает модель ***логистической регрессии (Logistic Regression).***

<a id="adequacy_models"></a>
## Проверьте модели на адекватность

Для оценки качества моделей и сравнения различных алгоритмов используем метрики ***confusion matrix, precision, recall, f1-score***.

Рассмотрим матрицы несоответствий (ошибок) ***confusion matrix*** для каждой модели:

#### Для модели решающее дерево:

In [26]:
pd.crosstab(target_test, test_predictions_dts)

col_0,0,1
is_ultra,,
0,421,26
1,110,86


#### Для модели случайный лес:

In [27]:
pd.crosstab(target_test, test_predictions_rfc)

col_0,0,1
is_ultra,,
0,416,31
1,105,91


#### Для модели логистическая регрессия:

In [28]:
pd.crosstab(target_test, test_predictions_lr)

col_0,0,1
is_ultra,,
0,443,4
1,187,9


Как мы видим модели ***Решающее дерево*** и ***Случайный лес*** имеют схожие матрицы несоответствий.  
Алгоритм ***Логистической регрессии*** зачастую ошибался и устанавливал метку класса 1 не верно(ошибка 2 рода), т.е. зачастую правильно не смог найти клиентов с тарифом **"Ультра"**.

#### Для каждой модели выведем метрики ***precision, recall, f1-score***.

**Характеристика метрик:**  

**precision** - указывает нам долю объектов, названных классификатором положительными и при этом действительно являющимися положительными.  

**recall** - показывает нам, какую долю объектов положительного класса из всех объектов положительного класса нашел алгоритм.  

**f1-score** - среднее гармоническое точности и полноты, она стремится к нулю, если точность или полнота стремится к нулю.

#### Для модели решающее дерево:

In [29]:
print(precision_score(target_test, test_predictions_dts))
print(recall_score(target_test, test_predictions_dts))
print(f1_score(target_test, test_predictions_dts))

0.7678571428571429
0.4387755102040816
0.5584415584415584


#### Для модели случайный лес:

In [30]:
print(precision_score(target_test, test_predictions_rfc))
print(recall_score(target_test, test_predictions_rfc))
print(f1_score(target_test, test_predictions_rfc))

0.7459016393442623
0.4642857142857143
0.5723270440251572


#### Для модели логистическая регрессия:

In [31]:
print(precision_score(target_test, test_predictions_lr))
print(recall_score(target_test, test_predictions_lr))
print(f1_score(target_test, test_predictions_lr))

0.6923076923076923
0.04591836734693878
0.0861244019138756


Метрики **recall_score и f1_score** самые низкие у модели **логистической регрессии**.  

Следовательно, наш вывод из матрицы несоответствий подтверждается, модель ***логистической регрессии*** имеет очень низкую способность выявления объектов класса 1 (тариф "Ультра").  


Модели ***Решающее дерево*** и ***Случайный лес*** имеют высокие оценки метрик, поэтому для построения системы, анализа поведения клиентов предлагаем использовать имеено эти модели.

#### Оценим сбалансированность данных:

In [32]:
df[df['is_ultra'] == 1]['is_ultra'].count() / len(df)

0.30647168637212197

Только около 30% данных принадлежат к классу 1 (тариф "Ультра").

Исходя из полученных результатов, можно сделать вывод, что для **оптимизации алгоритма моделей и повышения точности прогноза**, требуется уравновесить в данных количество пользователей с тарифами "Смарт" и "Ультра".

Создадим DataFrame с значениями признаков:
- равных нулю;
- с максимальными значениями признаков в исходных данных;
- с медианным значением признаков клиентов на тарифе "Ультра";
- с максимальными лимитами тарифа «Ультра»: 3000 минут разговора, 1000 сообщений и 30 Гб интернет-трафика.

In [33]:
df.max()

calls         244.00
minutes      1632.06
messages      224.00
mb_used     49745.73
is_ultra        1.00
dtype: float64

In [34]:
df[df['is_ultra']==1].median()

calls          74.00
minutes       502.55
messages       38.00
mb_used     19308.01
is_ultra        1.00
dtype: float64

In [35]:
new_df = pd.DataFrame(
    [[0, 0, 0, 0],
     [244, 1632, 224, 50000],
     [74, 503, 38, 19500],
     [800, 3000, 1000, 30720]],
    columns=features.columns)
new_df

,calls,minutes,messages,mb_used
0,0,0,0,0
1,244,1632,224,50000
2,74,503,38,19500
3,800,3000,1000,30720


Значение целевого признака в этой таблице должно быть 0 для первой строки и для всех остальных 1.  
Оценим результаты предсказаний моделей:

In [36]:
best_model_dts.predict(new_df)

array([1, 1, 0, 1])

In [37]:
best_model_rfc.predict(new_df)

array([1, 1, 0, 1])

In [38]:
model_lr.predict(new_df)

array([0, 0, 0, 0])

Как мы видими ни одна из моделей не выдала верный результат.  
Особо странно, что при нулевом значении всех ячеек, модели с лучшими значениями Accuracy выдают не верный результат.  
Модели Решающее дерево и Случайный лес ошибаются в 50% случаев, линейная регрессия в 75 % случаев.

#### Проверка на адекватность с помощью DummyClassifier

#### Вариант 1

In [39]:
X = [0, 1, 1, 1]
y = best_model_dts.predict(new_df)
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(X, y)
DummyClassifier(strategy="most_frequent")
dummy_clf.predict(X)
dummy_clf.score(X, y)

0.75

In [40]:
X = [0, 1, 1, 1]
y = best_model_rfc.predict(new_df)
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(X, y)
DummyClassifier(strategy="most_frequent")
dummy_clf.predict(X)
dummy_clf.score(X, y)

0.75

In [41]:
X = [0, 1, 1, 1]
y = model_lr.predict(new_df)
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(X, y)
DummyClassifier(strategy="most_frequent")
dummy_clf.predict(X)
dummy_clf.score(X, y)

1.0

#### Вариант 2

In [42]:
X = target
y = best_model_dts.predict(features)
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(X, y)
DummyClassifier(strategy="most_frequent")
dummy_clf.predict(X)
dummy_clf.score(X, y)

0.8226509023024269

In [43]:
X = target
y = best_model_rfc.predict(features)
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(X, y)
DummyClassifier(strategy="most_frequent")
dummy_clf.predict(X)
dummy_clf.score(X, y)

0.8105164903546982

In [44]:
X = target
y = model_lr.predict(features)
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(X, y)
DummyClassifier(strategy="most_frequent")
dummy_clf.predict(X)
dummy_clf.score(X, y)

0.9800871188550093

Вычислим долю значений равных 0 в целевом признаке:

In [45]:
1 - (sum(target) / len(target))

0.693528313627878

По итогу анализа на адекватность можно сказать, что предсказанные значения моделью линейной регрессии можно получить с такой же точностью случайно.

<a id="conclusion"></a>
## Вывод

Данные состоят из 5 колонок и 3214 строк, пропуски и дубликаты отсутствуют.  
**Известны следующие параметры:**
- *сalls* — количество звонков,
- *minutes* — суммарная длительность звонков в минутах,
- *messages* — количество sms-сообщений,
- *mb_used* — израсходованный интернет-трафик в Мб,
- *is_ultra* — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

Целевой признак в данных - столбец **is_ultra**.  

Для построения моделей данные были разбиты на 3 выборки (в соотношении 3:1:1): **обучающую, валидационную, тестовую.**  
    
Лучшим результат ***Accuracy (точности)*** в **0.783826** показала модель построенная по алгортму машинного обучения ***Random forest (Случайный лес)***.  

**Наиболее оптимальные гиперпараметры для модели** ***Random forest***:
- количество деревьев (n_estimators) - 5, 
- макисимальная глубина деревьев (max_depth) - 5, 
- число признаков (max_features) - 3.    


Все модели показали схожие результаты точности на тестовой выборке и при их обучении.  


| Model | Accuracy_valid | Accuracy_test |
| --- | --- | --- |
| Решающее дерево | 0.774495 | 0.788491 |
| Случайный лес | 0.783826 | 0.788491 |
| Логистическая регрессия | 0.696734 | 0.702955 |  


Наименьшую точность на тестовой выборке показала модель ***логистической регрессии (Logistic Regression).***  

**При анализе метрик моделей, получены следующие выводы:**    
1. Модели ***Решающее дерево*** и ***Случайный лес*** имеют схожие матрицы несоответствий;
2. Алгоритм ***Логистической регрессии*** зачастую ошибался и устанавливал метку класса 1 не верно(ошибка 2 рода);
3. Модель ***Логистической регрессии*** имеет очень низкую способность выявления объектов класса 1 (тариф "Ультра");  
4. Модели ***Решающее дерево*** и ***Случайный лес*** имеют высокие оценки метрик, поэтому для построения системы, анализа поведения клиентов предлагаем использовать имеено эти модели.
****
**Рекомендации:**  
- Требуется **уравновесить** в данных количество пользователей с тарифами "Смарт" и "Ультра", для оптимизации алгоритма моделей и повышения точности прогноза;
- Для дальнейшего построения системы классификации клиентов и выбору тарифов предлагаем использовать модели ***Случайный лес*** и ***Решающее дерево***.

